# Production model training with K-Folds cross-validation


This notebook shows how you can use the Evidently to:
* calculate prerformance and data drift for the model, performed as batch checks 
* log models quality & data drift using MLflow Tracking
* explore the result 

More examples are avaliable in the github: https://github.com/evidentlyai/evidently/tree/main/examples

Evidently docs: https://docs.evidentlyai.com/

Join our Discord: https://discord.com/invite/xZjKRaNp8b

In [120]:
import datetime
import joblib
import pandas as pd
import numpy as np
import requests
import zipfile
import io
import json

from pathlib import Path
from sklearn import datasets, ensemble, model_selection

from evidently.metrics import RegressionQualityMetric, RegressionErrorPlot, RegressionErrorDistribution
from evidently.metric_preset import DataDriftPreset, RegressionPreset
from evidently.pipeline.column_mapping import ColumnMapping
from evidently.report import Report

In [89]:
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

## Bicycle Demand Data

More information about the dataset can be found in UCI machine learning repository: https://archive.ics.uci.edu/ml/datasets/bike+sharing+dataset

Acknowledgement: Fanaee-T, Hadi, and Gama, Joao, 'Event labeling combining ensemble detectors and background knowledge', Progress in Artificial Intelligence (2013): pp. 1-15, Springer Berlin Heidelberg

In [90]:
# Load data saved in the previous step (in train_model.ipynb)

raw_data = pd.read_csv('../data/raw_data.csv', index_col=0)

In [91]:
# content = requests.get("https://archive.ics.uci.edu/static/public/275/bike+sharing+dataset.zip").content
# with zipfile.ZipFile(io.BytesIO(content)) as arc:
#     raw_data = pd.read_csv(arc.open("hour.csv"), header=0, sep=',', parse_dates=['dteday']) 
    
# raw_data.index = raw_data.apply(lambda row: datetime.datetime.combine(row.dteday.date(), datetime.time(row.hr)), axis=1)

In [92]:
raw_data.head()

,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
2011-01-01 00:00:00,1,2011-01-01,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0.0,3,13,16
2011-01-01 01:00:00,2,2011-01-01,1,0,1,1,0,6,0,1,0.22,0.2727,0.80,0.0,8,32,40
2011-01-01 02:00:00,3,2011-01-01,1,0,1,2,0,6,0,1,0.22,0.2727,0.80,0.0,5,27,32
2011-01-01 03:00:00,4,2011-01-01,1,0,1,3,0,6,0,1,0.24,0.2879,0.75,0.0,3,10,13
2011-01-01 04:00:00,5,2011-01-01,1,0,1,4,0,6,0,1,0.24,0.2879,0.75,0.0,0,1,1


# K-Folds split setup

## Model training 

In [93]:
target = 'cnt'
prediction = 'prediction'
numerical_features = ['temp', 'atemp', 'hum', 'windspeed', 'mnth', 'hr', 'weekday']
categorical_features = ['season', 'holiday', 'workingday', ]#'weathersit']

# Model Quality Evaluation (Prod)

In [94]:
import mlflow
#import mlflow.sklearn
from mlflow.tracking import MlflowClient

In [45]:
# from datetime import datetime

# datetime.strptime('2011-03-06 23:00:00', '%Y-%m-%d %H:%M:%S').isoweekday()

7

In [95]:
start_date_0 = '2011-01-02 00:00:00'
end_date_0 = '2011-01-30 23:00:00'

experiment_batches = [
    
    ('2011-01-31 00:00:00','2011-02-06 23:00:00'),
    ('2011-02-07 23:00:00','2011-02-13 23:00:00'),
    ('2011-02-14 23:00:00','2011-02-20 23:00:00'),
    ('2011-02-21 00:00:00','2011-02-27 23:00:00'),
    ('2011-02-28 00:00:00','2011-03-06 23:00:00'),  
]

In [96]:
reference = raw_data.loc[start_date_0:end_date_0]
print(reference.shape)



(640, 17)


In [97]:
column_mapping = ColumnMapping()

column_mapping.target = target
column_mapping.prediction = prediction
column_mapping.numerical_features = numerical_features
column_mapping.categorical_features = categorical_features

In [98]:
# # X_train, X_test, y_train, y_test = model_selection.train_test_split(
# #     reference[numerical_features + categorical_features],
# #     reference[target],
# #     test_size=0.3
# # )

# # preds_train = regressor.predict(X_train)
# # preds_test = regressor.predict(X_test)

# X_train['target'] = y_train
# X_train['prediction'] = preds_train

# X_test['target'] = y_test
# X_test['prediction'] = preds_test

In [121]:
REPORTS_DIR = '../reports'

model_path = Path('../models/model_get_started.joblib')
report_path = f"{REPORTS_DIR}/get_started_regression_performance_report.html"


In [122]:
from config import MLFLOW_TRACKING_URI


mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)
client = MlflowClient()
print(f"Client tracking uri: {client.tracking_uri}")

#set experiment
mlflow.set_experiment('Train Model - K-Fold')

ref_end_data = end_date_0

#start new run
for k, date in enumerate(experiment_batches):
    
    print(k, date[0],  date[1])
    
        
    print(f"train_end_data: {ref_end_data}") 
    X_train = raw_data.loc[start_date_0:ref_end_data, numerical_features + categorical_features]
    y_train = raw_data.loc[start_date_0:ref_end_data, target]
    print(X_train.shape, y_train.shape)
    
    current = raw_data.loc[date[0]:date[1]]
    X_test = current.loc[:, numerical_features + categorical_features]
    y_test = current[target]
    print(X_test.shape, y_test.shape)
    
    # Update reference end date
    ref_end_data = date[1]
    
    regressor = ensemble.RandomForestRegressor(random_state = 0, n_estimators = 50)
    regressor.fit(X_train, y_train)
    
    ref_prediction = regressor.predict(reference[numerical_features + categorical_features])
    reference['prediction'] = ref_prediction
    
    prediction = regressor.predict(current[numerical_features + categorical_features])
    current['prediction'] = prediction
    
    
    # Get metrics
    regression_performance_report = Report(metrics=[
        RegressionQualityMetric(),
    ])
    regression_performance_report.run(
        reference_data=reference, 
        current_data=current,
        column_mapping=column_mapping)
    # logged_json = json.loads(the_report.json())
    
    train_report_metrics = regression_performance_report.as_dict()
    me = train_report_metrics['metrics'][0]['result']['current']['mean_error']
    mae = train_report_metrics['metrics'][0]['result']['current']["mean_abs_error"]
    
    # Save model
    joblib.dump(regressor, model_path)
    regression_performance_report.save_html(report_path)

    
    with mlflow.start_run() as run: #inside brackets run_name='test'
        
        # Show newly created run metadata info
        print("Experiment id: {}".format(run.info.experiment_id))
        print("Run id: {}".format(run.info.run_id))
        print("Run name: {}".format(run.info.run_name))
        print('MLFlow tracking uri:', mlflow.get_tracking_uri())
        print('MLFlow artifact uri:', mlflow.get_artifact_uri())
        
        # Log parameters
        mlflow.log_param("begin", date[0])
        mlflow.log_param("end", date[1])
        
        # Log metrics
        mlflow.log_metric('me', round(me, 3))
        mlflow.log_metric('mae', round(mae, 3))
        
        # Log model 
        mlflow.log_artifact(model_path)
        
        # Log the regression_performance_report as an artifact
        mlflow.log_artifact(report_path)

Client tracking uri: http://localhost:5000
0 2011-01-31 00:00:00 2011-02-06 23:00:00
train_end_data: 2011-01-30 23:00:00
(640, 10) (640,)
(165, 10) (165,)
Experiment id: 155405333588197577
Run id: 20085be7ec974cf4aa226269002d01a4
Run name: amazing-ox-272
MLFlow tracking uri: http://localhost:5000
MLFlow artifact uri: mlflow-artifacts:/155405333588197577/20085be7ec974cf4aa226269002d01a4/artifacts
1 2011-02-07 23:00:00 2011-02-13 23:00:00
train_end_data: 2011-02-06 23:00:00
(805, 10) (805,)
(140, 10) (140,)
Experiment id: 155405333588197577
Run id: 24e4ddbe585c426dbae96f2129ea04cf
Run name: wise-snail-611
MLFlow tracking uri: http://localhost:5000
MLFlow artifact uri: mlflow-artifacts:/155405333588197577/24e4ddbe585c426dbae96f2129ea04cf/artifacts
2 2011-02-14 23:00:00 2011-02-20 23:00:00
train_end_data: 2011-02-13 23:00:00
(968, 10) (968,)
(142, 10) (142,)
Experiment id: 155405333588197577
Run id: 854eeb0390ea46f19ef375928685dab5
Run name: big-ox-64
MLFlow tracking uri: http://localhost:

## Nested Runs 

In [133]:
from config import MLFLOW_TRACKING_URI


mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)
client = MlflowClient()
mlflow.set_experiment('Train Model - K-Fold')
print(f"Client tracking uri: {client.tracking_uri}")

ref_end_data = end_date_0

# Start a new Run (Parent Run)
with mlflow.start_run() as run: 
    
    
    metrics = {}

    #start new run
    for k, date in enumerate(experiment_batches):
        
        print(k, date[0],  date[1])
            
        print(f"train_end_data: {ref_end_data}") 
        X_train = raw_data.loc[start_date_0:ref_end_data, numerical_features + categorical_features]
        y_train = raw_data.loc[start_date_0:ref_end_data, target]
        print(X_train.shape, y_train.shape)
        
        current = raw_data.loc[date[0]:date[1]]
        X_test = current.loc[:, numerical_features + categorical_features]
        y_test = current[target]
        print(X_test.shape, y_test.shape)
        
        # Update reference end date
        ref_end_data = date[1]
        
        regressor = ensemble.RandomForestRegressor(random_state = 0, n_estimators = 50)
        regressor.fit(X_train, y_train)
        
        ref_prediction = regressor.predict(reference[numerical_features + categorical_features])
        reference['prediction'] = ref_prediction
        
        prediction = regressor.predict(current[numerical_features + categorical_features])
        current['prediction'] = prediction
        
        
        # Get metrics
        regression_performance_report = Report(metrics=[
            RegressionQualityMetric(),
        ])
        regression_performance_report.run(
            reference_data=reference, 
            current_data=current,
            column_mapping=column_mapping)
        # logged_json = json.loads(the_report.json())
        
        train_report_metrics = regression_performance_report.as_dict()
        me = train_report_metrics['metrics'][0]['result']['current']['mean_error']
        mae = train_report_metrics['metrics'][0]['result']['current']["mean_abs_error"]
        metrics.update({date[1]: {'me': me, 'mae': mae}})
        
        # Save model
        joblib.dump(regressor, model_path)
        regression_performance_report.save_html(report_path)
        
        # Run a Childe Run for each Fold 
        with mlflow.start_run(run_name=date[1], 
                              nested=True,
                            #   experiment_id=experiment_id
                              ) as child_run:
            
            # Show newly created run metadata info
            print("Experiment id: {}".format(run.info.experiment_id))
            print("Run id: {}".format(run.info.run_id))
            print("Run name: {}".format(run.info.run_name))
            print('MLFlow tracking uri:', mlflow.get_tracking_uri())
            print('MLFlow artifact uri:', mlflow.get_artifact_uri())
            
            # Log parameters
            mlflow.log_param("begin", date[0])
            mlflow.log_param("end", date[1])
            
            # Log metrics
            mlflow.log_metric('me', round(me, 3))
            mlflow.log_metric('mae', round(mae, 3))
            
            # Log the regression_performance_report as an artifact
            mlflow.log_artifact(report_path)
            
        # Log the last batch model as the parent Run model
        mlflow.log_artifact(model_path)
        
        # Log metrics
        average_run_merics = pd.DataFrame.from_dict(metrics).T.mean().round(3).to_dict()
        mlflow.log_metrics(average_run_merics )

Client tracking uri: http://localhost:5000
0 2011-01-31 00:00:00 2011-02-06 23:00:00
train_end_data: 2011-01-30 23:00:00
(640, 10) (640,)
(165, 10) (165,)
Experiment id: 155405333588197577
Run id: 176ad60894914599aa7b15711370d711
Run name: receptive-mare-831
MLFlow tracking uri: http://localhost:5000
MLFlow artifact uri: mlflow-artifacts:/155405333588197577/66729c5de51048debb93ac531c1b49ec/artifacts
1 2011-02-07 23:00:00 2011-02-13 23:00:00
train_end_data: 2011-02-06 23:00:00
(805, 10) (805,)
(140, 10) (140,)
Experiment id: 155405333588197577
Run id: 176ad60894914599aa7b15711370d711
Run name: receptive-mare-831
MLFlow tracking uri: http://localhost:5000
MLFlow artifact uri: mlflow-artifacts:/155405333588197577/24701cfccc7444ca9ce831d16b378d7a/artifacts
2 2011-02-14 23:00:00 2011-02-20 23:00:00
train_end_data: 2011-02-13 23:00:00
(968, 10) (968,)
(142, 10) (142,)
Experiment id: 155405333588197577
Run id: 176ad60894914599aa7b15711370d711
Run name: receptive-mare-831
MLFlow tracking uri: 

In [127]:
metrics

{'2011-02-06 23:00:00': {'me': -7.015272727272727, 'mae': 13.564848484848484},
 '2011-02-13 23:00:00': {'me': -4.623285714285714, 'mae': 10.085571428571429},
 '2011-02-20 23:00:00': {'me': -15.03380281690141, 'mae': 23.230140845070423},
 '2011-02-27 23:00:00': {'me': 0.8769620253164568, 'mae': 22.516455696202527},
 '2011-03-06 23:00:00': {'me': 4.758545454545455, 'mae': 19.815272727272724}}

In [132]:
pd.DataFrame.from_dict(metrics).T.mean().round(3).to_dict()

{'me': -4.207, 'mae': 17.842}